In [1]:
import os
import json
import datetime
import plotly.express as px
import pandas as pd

In [2]:
path = 'data/instagram/'
bgeorgakas_path = path + 'instagram-bgeorgakas-2024-04-11-9ywghq7K/your_instagram_activity/'
about404_path = path + 'instagram-4b0ut404-2024-04-12-xkfnNfXv/your_instagram_activity'


In [3]:
def get_actions(path: str, reel_comments = True) -> list:
    with open(path + '/content/posts_1.json', 'r') as file:
        data = json.load(file)

    post_timestamps = []
    for chunk in data:
        for post in chunk['media']:
            post_timestamps.append(datetime.datetime.fromtimestamp(post['creation_timestamp'])) 

    with open(path + '/content/stories.json', 'r') as file:
        data = json.load(file)

    stories_timestamps = []
    for story in data['ig_stories']:
        stories_timestamps.append(datetime.datetime.fromtimestamp(story['creation_timestamp']))
    
    with open(path + '/comments/post_comments_1.json', 'r') as file:
        data = json.load(file)

    post_comments_timestamps = []
    for comment in data:
        post_comments_timestamps.append(
            datetime.datetime.fromtimestamp(
                comment['string_map_data']['Time']['timestamp']
            )
        )

    reels_comments_timestamps = []
    if reel_comments:
        with open(path + '/comments/reels_comments.json', 'r') as file:
            data = json.load(file)

        
        for comment in data['comments_reels_comments']:
            reels_comments_timestamps.append(
                datetime.datetime.fromtimestamp(
                    comment['string_map_data']['Time']['timestamp']
                )
            )

    return post_timestamps + stories_timestamps + post_comments_timestamps + reels_comments_timestamps
    

In [4]:
bgeorgakas_actions = get_actions(bgeorgakas_path)
about404_actions = get_actions(about404_path, reel_comments = False)

In [5]:
actions = bgeorgakas_actions + about404_actions
actions_df = pd.DataFrame(data = actions, columns = ['date_time'])
actions_df = actions_df.assign(date = actions_df.date_time.dt.date)

In [6]:
actions_df.sort_values(by = 'date_time').to_csv('data/instagram_actions.csv')